Model: `tts_models/en/vctk/vits`
Speakers:
- p286: Low breathy male
- p287: Very deep male

In [10]:
import os

# BaseDatasetConfig: defines name, formatter and path of the dataset.
from TTS.tts.configs.shared_configs import BaseDatasetConfig

output_path = "tts_train_dir"
if not os.path.exists(output_path):
    os.makedirs(output_path)
    
dataset_config = BaseDatasetConfig(
    formatter="ljspeech", meta_file_train="metadata.txt", path="dataset"
)

In [11]:
from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.models.vits import Vits, VitsAudioConfig

audio_config = VitsAudioConfig(
    sample_rate=22050, win_length=1024, hop_length=256, num_mels=80, mel_fmin=0, mel_fmax=None
)

config = VitsConfig(
    audio=audio_config,
    run_name="vits_ljspeech",
    batch_size=32,
    eval_batch_size=16,
    batch_group_size=5,
    num_loader_workers=8,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=1000,
    text_cleaner="english_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    compute_input_seq_cache=True,
    print_step=25,
    print_eval=True,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    cudnn_benchmark=False,
)

In [12]:
# INITIALIZE THE AUDIO PROCESSOR
# Audio processor is used for feature extraction and audio I/O.
# It mainly serves to the dataloader and the training loggers.

from TTS.utils.audio import AudioProcessor
ap = AudioProcessor.init_from_config(config)

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


In [13]:
# INITIALIZE THE TOKENIZER
# Tokenizer is used to convert text to sequences of token IDs.
# config is updated with the default characters if not defined in the config.

from TTS.tts.utils.text.tokenizer import TTSTokenizer
tokenizer, config = TTSTokenizer.init_from_config(config)

In [15]:
# LOAD DATA SAMPLES
# Each sample is a list of ```[text, audio_file_path, speaker_name]```
# You can define your custom sample loader returning the list of samples.
# Or define your custom formatter and pass it to the `load_tts_samples`.
# Check `TTS.tts.datasets.load_tts_samples` for more details.

from TTS.tts.datasets import load_tts_samples
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

 | > Found 131 files in C:\Users\iambl\GitHub\TTS\notebooks\worgen\dataset


In [16]:
# init model
model = Vits(config, ap, tokenizer, speaker_manager=None)

In [18]:
# init the trainer and 🚀
from trainer import Trainer, TrainerArgs
model_path = "C:\\Users\\iambl\\AppData\\Local\\tts\\tts_models--en--vctk--vits\\model_file.pth"
trainer = Trainer(
    TrainerArgs(restore_path=model_path),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
)
trainer.fit()

 > Training Environment:
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 16
 | > Num. of Torch Threads: 8
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 > Start Tensorboard: tensorboard --logdir=tts_train_dir\vits_ljspeech-February-17-2023_08+08PM-334a8192
 > Restoring from model_file.pth ...
 > Restoring Model...
 > Partial model initialization...
 | > Layer missing in the model definition: emb_g.weight
 | > Layer missing in the model definition: posterior_encoder.enc.cond_layer.bias
 | > Layer missing in the model definition: posterior_encoder.enc.cond_layer.weight_g
 | > Layer missing in the model definition: posterior_encoder.enc.cond_layer.weight_v
 | > Layer missing in the model definition: flow.flows.0.enc.cond_layer.bias
 | > Layer missing in the model definition: flow.flows.0.enc.cond_layer.weight_g
 | > Layer missing in the model definition: flow.flows.0.enc.cond_layer.weight_v
 | > Layer m

[*] Pre-computing phonemes...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 130/130 [00:05<00:00, 21.82it/s]

 > TRAINING (2023-02-17 20:09:06) 




> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: en-us
		| > phoneme backend: espeak
| > Number of instances : 130
 | > Preprocessing samples
 | > Max text length: 144
 | > Min text length: 4
 | > Avg text length: 24.515384615384615
 | 
 | > Max audio length: 233073.0
 | > Min audio length: 11377.0
 | > Avg audio length: 43634.64615384615
 | > Num. instances discarded samples: 0
 | > Batch group size: 160.


C:\Users\iambl\GitHub\TTS\venv\lib\site-packages\torch\functional.py:580: UserWarning: stft will soon require the return_complex parameter be given for real inputs, and will further require that return_complex=True in a future PyTorch release. (Triggered internally at  ..\aten\src\ATen\native\SpectralOps.cpp:639.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:/Users/iambl/GitHub/TTS/notebooks/worgen/tts_train_dir/vits_ljspeech-February-17-2023_08+08PM-334a8192\\events.out.tfevents.1676682537.AMIT-PC'